In [1]:
import numpy as np
import gurobipy as gbp
import time
import pandas as pd
np.random.seed(352)


In [58]:
n_com=2 #no of commodities
n_sup=2 #no of supply nodes
n_dem=3 #no of demands nodes

In [59]:
#Supply Demand of all commodities 
"""[
[Day1:[Supplier_1:comm1,Supplier_1:comm2],[s2c1,s2c2]],
Day2:[],
Day3:[]
    ]"""
# Si=np.array([[[100,200],[200,700]],
#              [[300,100],[500,200]],
#              [[600,200],[1000,400]]])
# Dj=np.array([[[75,290],[150,420],[200,325]],
#              [[290,150],[200,400],[470,250]],
#              [[580,300],[400,800],[940,500]]])

Si=np.array([[[100,200],[200,700]],
             [[300,100],[500,200]],
             [[400,300],[500,400]]])
Dj=np.array([[[75,190],[150,320],[200,225]],
             [[120,150],[100,400],[300,250]],
             [[300,250],[400,900],[600,1500]]])
Si.shape,Dj.shape #days X nodes X commodities

((3, 2, 2), (3, 3, 2))

In [60]:
# check assumption that demand is always higher than supply
np.sum(Si,axis=1)<np.sum(Dj,axis=1)

array([[ True, False],
       [False,  True],
       [ True,  True]])

In [61]:
#Vehicle capacity, number of vehicles with each supplier, cost of transportation from supply nodes to demand nodes
V_cap=150
V_num=np.array([5,7])
V_cost=np.array([[10,15,20],[18,13,20]])

In [62]:
np.sum(Si,axis=2)<[[V_cap*V_num]*3]

array([[[ True,  True],
        [ True,  True],
        [ True,  True]]])

In [63]:
#People Variables
n_pep=np.array([[[25,25],[50,50],[30,30]],
                [[25,25],[50,50],[30,30]],
                [[25,25],[50,50],[30,30]]]) #no of people on each demand node,
mu=[4,10] #consumption rate (comm req per person per day)
pc=[0.1,0.9]

In [64]:
n_days=len(Si)
days_range=range(n_days)
days_range

range(0, 3)

In [65]:
# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
comm_range=range(n_com)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)

print (supply_nodes_range, demand_nodes_range,comm_range, all_nodes_len)

range(0, 2) range(0, 3) range(0, 2) 6


In [66]:
pc=[0.7,0.3]

In [74]:
Pc=np.array([Dj[ix]/Dj[ix].sum(axis=0) for ix in range(len(Dj))])
Pc*pc

array([[[0.12352941, 0.07755102],
        [0.24705882, 0.13061224],
        [0.32941176, 0.09183673]],

       [[0.16153846, 0.05625   ],
        [0.13461538, 0.15      ],
        [0.40384615, 0.09375   ]],

       [[0.16153846, 0.02830189],
        [0.21538462, 0.10188679],
        [0.32307692, 0.16981132]]])

In [76]:
dep_time=24
dep_cost=np.exp(1.5031+0.1172*dep_time)-np.exp(1.5031)
dep_cost

70.38538215041021

In [77]:
np.exp(1.5031+0.1172*48)-np.exp(1.5031)

1242.7588778369152

([[[0, 0], [0, 0], [0, 0]]], [array([[0, 0],
         [0, 0]])])

In [120]:
unserved_people=[[[0,0],[0,0],[0,0]]]
prev_inventory=[Si[0]*0]
unserved_people,prev_inventory
cols=['Day','S_Node']+['D'+str(i+1)+'_c'+str(j+1) for i in demand_nodes_range for j in comm_range]
cols+=['Total_S_c'+str(i+1) for i in comm_range]
cols+=['Deliver_S_c'+str(i+1) for i in comm_range]
cols+=['D'+str(i+1)+'_v' for i in demand_nodes_range]
cols+=['Total_V']

df=pd.DataFrame(columns=cols,index=None,dtype=int)

cols2=['Day','D_Node']+['Total'+'_c'+str(i+1) for i in comm_range]+['Unserv'+'_c'+str(i+1) for i in comm_range]
df2=pd.DataFrame(columns=cols2,index=None,dtype=int)
df2

unserved_people=[[[0,0],[0,0],[0,0]]]
for day in days_range:
    print ('#'*50,'  Day',day+1,'  ','#'*50)
    # Create Model, Set MIP Focus, Add Variables, & Update Model
    m = gbp.Model(' -- The Multi Commodity Vehicle Transportation Problem -- ')

    # Set MIP Focus to 2 for optimality
    m.setParam('MIPFocus', 2)
    # m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
    # m.setParam(gbp.GRB.Param.PoolGap, 0.10)

    decision_var = []
    vehicles_var=[]
    unserved_var=[]
    for orig in supply_nodes_range:
        decision_var.append([])
        vehicles_var.append([])
        for dest in demand_nodes_range:
            decision_var[orig].append([])
            vehicles_var[orig].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                              name='S'+str(orig+1)+'_D'+str(dest+1)+'_V'))
            for comm in comm_range:
    #             print (comm,decision_var)
                decision_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER, 
    #                                         obj=Cij[orig][dest],
    #                                            obj=1,
                                            name='S'+str(orig+1)+'_D'+str(dest+1)+'_c'+str(comm+1)))
    for dest in demand_nodes_range:
        unserved_var.append([])
        for comm in comm_range:
            unserved_var[dest].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                              name='D'+str(dest+1)+'_c'+str(comm+1)+'_U'))
    # Update Model Variables
    m.update() 
    
    #sum(sum[(Demand - net supplied)*priority for every demand node] for every commodity)
    first_term=10*gbp.quicksum(gbp.quicksum((int(Dj[day][dest][comm])-gbp.quicksum(decision_var[orig][dest][comm] for orig in supply_nodes_range))*(Pc[day][dest][comm])
                                for dest in demand_nodes_range) for comm in comm_range)
    print ('First term: ',first_term)
    
    second_term=gbp.quicksum(gbp.quicksum(vehicles_var[orig][dest]*V_cost[orig][dest] for dest in demand_nodes_range) for orig in supply_nodes_range)
    third_term=0.1*gbp.quicksum(gbp.quicksum(unserved_var[dest][comm]*dep_cost for comm in comm_range) for dest in demand_nodes_range)
    
    #objective function
    m.setObjective(first_term+second_term+third_term,gbp.GRB.MINIMIZE)

    m.update()

#     Add Supply Constraints
    for orig in supply_nodes_range:
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                     for dest in demand_nodes_range) - Si[day][orig][comm] - prev_inventory[day][orig][comm] <= 0)
    # Add Demand Constraints
    for dest in demand_nodes_range:  
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm] 
                                     for orig in supply_nodes_range) - Dj[day][dest][comm] <= 0)
    #Add vehicle constraints
#     for orig in supply_nodes_range:
#         m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
#                                  for dest in demand_nodes_range for comm in comm_range) - V_cap*V_num[orig] <=0)

    for orig in supply_nodes_range:
        m.addConstr(gbp.quicksum(vehicles_var[orig][dest] for dest in demand_nodes_range) - V_num[orig] <=0)

    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr(-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/V_cap + vehicles_var[orig][dest]>=0)
    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr(-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/V_cap + vehicles_var[orig][dest]<=1)

    ######Add unserved people contstraints
    #unserved people at t <= num of people at t + unserved people at t-1
    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(unserved_var[dest][comm]<=n_pep[day][dest][comm]+unserved_people[day][dest][comm])
            
    # supplied commodity at demand node > (no of people at t + unserved people at t-1 - unserved people at t)*consumption rate
    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(sum(decision_var[orig][dest][comm] for orig in supply_nodes_range)-
                        ((n_pep[day][dest][comm]+unserved_people[day][dest][comm]-unserved_var[dest][comm])*mu[comm])>=0)
            
        
#      Adding 0 inventory for next day constraint: no supply left for t+1
#     for orig in supply_nodes_range:
#         for comm in comm_range:
#             m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
#                                      for dest in demand_nodes_range) - Si[day][orig][comm] >= 0)

    #  Optimize and Print( Results)
    m.optimize()
    m.write('./path.lp')
    print (m.display())

#     m.display()
    prev_inventory.append([])

    for orig in supply_nodes_range:
        prev_inventory[day+1].append([])
        for comm in comm_range:
            prev_inventory[day+1][orig].append(Si[day][orig][comm]+prev_inventory[day][orig][comm]-sum(decision_var[orig][dest][comm].x
                                 for dest in demand_nodes_range))

#     print (prev_inventory)
    unserved_people.append([])
    for dest in demand_nodes_range:
        unserved_people[day+1].append([])
        for comm in comm_range:
            unserved_people[day+1][dest].append(unserved_var[dest][comm].x)

            
    
    #Populate DataFrame
    for ix in supply_nodes_range:
        r_t=[]
        r_t+=[decision_var[ix][iy][iz].x for iy in demand_nodes_range for iz in comm_range]
        r_t+=[Si[day][ix][iy] for iy in comm_range]
        r_t+=[sum(decision_var[ix][iy][iz].x for iy in demand_nodes_range) for iz in comm_range]
        r_t+=[vehicles_var[ix][iy].x for iy in demand_nodes_range]
        r_t+=[V_num[ix]]
        df.loc[len(df)]=[day+1,ix+1]+r_t

    for ix in demand_nodes_range:
        r_t=[]
        r_t+=[n_pep[day][ix][iy]+unserved_people[day][ix][iy] for iy in comm_range]
        r_t+=[unserved_var[ix][iy].x for iy in comm_range]
        df2.loc[len(df2)]=[day+1,int(ix+1)]+r_t
    
    first_term_val=sum(sum((int(Dj[day][dest][comm])-sum(decision_var[orig][dest][comm].x for orig in supply_nodes_range))*(Pc[day][dest][comm])
                                for dest in demand_nodes_range) for comm in comm_range)
    second_term_val=sum(sum(vehicles_var[orig][dest].x*V_cost[orig][dest] for dest in demand_nodes_range) for orig in supply_nodes_range)
    third_term_val=0.1*sum(sum(unserved_var[dest][comm].x*dep_cost for comm in comm_range) for dest in demand_nodes_range)
    
    print ('^'*100)
    print ("First term: ",int(first_term_val)," Second term: ",int(second_term_val)," Third term: ",int(third_term_val))
    print ('^'*100)


##################################################   Day 1    ##################################################
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
First term:  <gurobi.LinExpr: 4176.070428171269 + -1.7647058823529413 S1_D1_c1 + -1.7647058823529413 S2_D1_c1 + -3.5294117647058827 S1_D2_c1 + -3.5294117647058827 S2_D2_c1 + -4.705882352941177 S1_D3_c1 + -4.705882352941177 S2_D3_c1 + -2.5850340136054424 S1_D1_c2 + -2.5850340136054424 S2_D1_c2 + -4.353741496598639 S1_D2_c2 + -4.353741496598639 S2_D2_c2 + -3.0612244897959187 S1_D3_c2 + -3.0612244897959187 S2_D3_c2>
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 36 rows, 24 columns and 90 nonzeros
Model fingerprint: 0xab9d43e2
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective

Best objective 2.067309885116e+03, best bound 2.067309885116e+03, gap 0.0000%
Minimize
   <gurobi.LinExpr: 5262.5 + 10.0 S1_D1_V + -2.307692307692308 S1_D1_c1 + -1.875 S1_D1_c2 + 15.0 S1_D2_V + -1.9230769230769231 S1_D2_c1 + -5.0 S1_D2_c2 + 20.0 S1_D3_V + -5.769230769230768 S1_D3_c1 + -3.125 S1_D3_c2 + 18.0 S2_D1_V + -2.307692307692308 S2_D1_c1 + -1.875 S2_D1_c2 + 13.0 S2_D2_V + -1.9230769230769231 S2_D2_c1 + -5.0 S2_D2_c2 + 20.0 S2_D3_V + -5.769230769230768 S2_D3_c1 + -3.125 S2_D3_c2 + 7.038538215041021 D1_c1_U + 7.038538215041021 D1_c2_U + 7.038538215041021 D2_c1_U + 7.038538215041021 D2_c2_U + 7.038538215041021 D3_c1_U + 7.038538215041021 D3_c2_U>
Subject To
   R0 : <gurobi.LinExpr: S1_D1_c1 + S1_D2_c1 + S1_D3_c1> <= 300.0
   R1 : <gurobi.LinExpr: S1_D1_c2 + S1_D2_c2 + S1_D3_c2> <= 110.0
   R2 : <gurobi.LinExpr: S2_D1_c1 + S2_D2_c1 + S2_D3_c1> <= 500.0
   R3 : <gurobi.LinExpr: S2_D1_c2 + S2_D2_c2 + S2_D3_c2> <= 355.0
   R4 : <gurobi.LinExpr: S1_D1_c1 + S2_D1_c1> <= 120.0
   R5 : <gu

   R18 : <gurobi.LinExpr: S1_D1_V + -0.006666666666666667 S1_D1_c1 + -0.006666666666666667 S1_D1_c2> <= 1.0
   R19 : <gurobi.LinExpr: S1_D2_V + -0.006666666666666667 S1_D2_c1 + -0.006666666666666667 S1_D2_c2> <= 1.0
   R20 : <gurobi.LinExpr: S1_D3_V + -0.006666666666666667 S1_D3_c1 + -0.006666666666666667 S1_D3_c2> <= 1.0
   R21 : <gurobi.LinExpr: S2_D1_V + -0.006666666666666667 S2_D1_c1 + -0.006666666666666667 S2_D1_c2> <= 1.0
   R22 : <gurobi.LinExpr: S2_D2_V + -0.006666666666666667 S2_D2_c1 + -0.006666666666666667 S2_D2_c2> <= 1.0
   R23 : <gurobi.LinExpr: S2_D3_V + -0.006666666666666667 S2_D3_c1 + -0.006666666666666667 S2_D3_c2> <= 1.0
   R24 : <gurobi.LinExpr: D1_c1_U> <= 45.0
   R25 : <gurobi.LinExpr: D1_c2_U> <= 56.0
   R26 : <gurobi.LinExpr: D2_c1_U> <= 93.0
   R27 : <gurobi.LinExpr: D2_c2_U> <= 78.0
   R28 : <gurobi.LinExpr: D3_c1_U> <= 30.0
   R29 : <gurobi.LinExpr: D3_c2_U> <= 62.0
   R30 : <gurobi.LinExpr: S1_D1_c1 + S2_D1_c1 + 4.0 D1_c1_U> >= 180.0
   R31 : <gurobi.LinExpr

In [121]:
prev_inventory

[array([[0, 0],
        [0, 0]]),
 [[0.0, 10.0], [0.0, 155.0]],
 [[140.0, 0.0], [140.0, 0.0]],
 [[90.0, 0.0], [0.0, 0.0]]]

In [122]:
unserved_people

[[[0, 0], [0, 0], [0, 0]],
 [[25.0, 6.0], [18.0, 18.0], [-0.0, 8.0]],
 [[20.0, 31.0], [43.0, 28.0], [-0.0, 32.0]],
 [[8.0, 56.0], [-0.0, 74.0], [-0.0, -0.0]]]

In [123]:
unserved_people[1][2][1]

8.0

In [124]:
df

,Day,S_Node,D1_c1,D1_c2,D2_c1,D2_c2,D3_c1,D3_c2,Total_S_c1,Total_S_c2,Deliver_S_c1,Deliver_S_c2,D1_v,D2_v,D3_v,Total_V
0,1.0,1.0,-0.0,190.0,-0.0,-0.0,100.0,-0.0,100.0,200.0,100.0,190.0,2.0,-0.0,1.0,5.0
1,1.0,2.0,-0.0,0.0,128.0,320.0,72.0,225.0,200.0,700.0,200.0,545.0,0.0,3.0,2.0,7.0
2,2.0,1.0,120.0,-0.0,40.0,110.0,0.0,0.0,300.0,100.0,160.0,110.0,1.0,1.0,0.0,5.0
3,2.0,2.0,0.0,-0.0,60.0,290.0,300.0,65.0,500.0,200.0,360.0,355.0,0.0,3.0,3.0,7.0
4,3.0,1.0,150.0,-0.0,-0.0,-0.0,300.0,300.0,400.0,300.0,450.0,300.0,1.0,0.0,4.0,5.0
5,3.0,2.0,-0.0,-0.0,400.0,40.0,240.0,360.0,500.0,400.0,640.0,400.0,0.0,3.0,4.0,7.0


In [125]:
df2

,Day,D_Node,Total_c1,Total_c2,Unserv_c1,Unserv_c2
0,1.0,1.0,25.0,25.0,25.0,6.0
1,1.0,2.0,50.0,50.0,18.0,18.0
2,1.0,3.0,30.0,30.0,-0.0,8.0
3,2.0,1.0,50.0,31.0,20.0,31.0
4,2.0,2.0,68.0,68.0,43.0,28.0
5,2.0,3.0,30.0,38.0,-0.0,32.0
6,3.0,1.0,45.0,56.0,8.0,56.0
7,3.0,2.0,93.0,78.0,-0.0,74.0
8,3.0,3.0,30.0,62.0,-0.0,-0.0


In [109]:
n_pep

array([[[25, 25],
        [50, 50],
        [30, 30]],

       [[25, 25],
        [50, 50],
        [30, 30]],

       [[25, 25],
        [50, 50],
        [30, 30]]])

In [40]:
Dj

array([[[  75,  290],
        [ 150,  420],
        [ 200,  325]],

       [[ 290,  150],
        [ 200,  400],
        [ 470,  250]],

       [[ 300,  250],
        [ 400,  900],
        [ 600, 1500]]])